In [6]:
%%file demo_3d.py

from __future__ import print_function, division
import os
import sys, petsc4py
petsc4py.init(sys.argv)
import mpi4py.MPI as mpi
from petsc4py import PETSc
import numpy as np
from elasticity import *

def rhs(coords, rhs):
    rhs[..., 1] = -9.81# + rand

OptDB = PETSc.Options()
Lx = OptDB.getInt('Lx', 10)
Ly = OptDB.getInt('Ly', 1)
Lz = OptDB.getInt('Lz', 1)
n  = OptDB.getInt('n', 16)
nx = OptDB.getInt('nx', Lx*n)
ny = OptDB.getInt('ny', Ly*n)
nz = OptDB.getInt('nz', Lz*n)
E1 = OptDB.getReal('E1', 10**6)
E2 = OptDB.getReal('E2', 1)
nu1 = OptDB.getReal('nu1', 0.4)
nu2 = OptDB.getReal('nu2', 0.4)

hx = Lx/(nx - 1)
hy = Ly/(ny - 1)
hz = Lz/(nz - 1)
h = [hx, hy, hz]

da = PETSc.DMDA().create([nx, ny, nz], dof=3, stencil_width=1)
da.setUniformCoordinates(xmax=Lx, ymax=Ly, zmax=Lz)
da.setMatType(PETSc.Mat.Type.IS)

path = './output_3d/'
if mpi.COMM_WORLD.rank == 0:
    if not os.path.exists(path):
        os.mkdir(path)
    else:
        os.system('rm {}/*.vts'.format(path))

class callback:
    def __init__(self, da):
        self.da = da
        ranges = da.getRanges()
        ghost_ranges = da.getGhostRanges()
        
        self.slices = []
        for r, gr in zip(ranges, ghost_ranges):
            self.slices.append(slice(gr[0], r[1]))
        self.slices = tuple(self.slices)

        self.it = 0

    def __call__(self, locals):
        pyKSP = locals['self']
        proj = pyKSP.mpc.proj


        if self.it == 0:
            work, _ = proj.A.getVecs()
            for i, vec in enumerate(proj.coarse_vecs):
                if vec:
                    proj.workl = vec.copy()
                else:
                    proj.workl.set(0.)
                work.set(0)
                proj.scatter_l2g(proj.workl, work, PETSc.InsertMode.ADD_VALUES)

                viewer = PETSc.Viewer().createVTK(path + 'coarse_vec_{}.vts'.format(i), 'w', comm = PETSc.COMM_WORLD)
                tmp = self.da.createGlobalVec()
                tmpl_a = self.da.getVecArray(tmp)
                work_a = self.da.getVecArray(work)
                tmpl_a[:] = work_a[:]
                tmp.view(viewer)
                viewer.destroy()
            self.it += 1

def lame_coeff(x, y, z, v1, v2):
    output = np.empty(x.shape)
    mask = np.logical_or(np.logical_and(.2<=z, z<=.4),np.logical_and(.6<=z, z<=.8))
    output[mask] = v1
    output[np.logical_not(mask)] = v2
    return output

# non constant Young's modulus and Poisson's ratio 
E = buildCellArrayWithFunction(da, lame_coeff, (E1,E2))
nu = buildCellArrayWithFunction(da, lame_coeff, (nu1, nu2))

lamb = (nu*E)/((1+nu)*(1-2*nu)) 
mu = .5*E/(1+nu)

x = da.createGlobalVec()
b = buildRHS(da, h, rhs)
A = buildElasticityMatrix(da, h, lamb, mu)
A.assemble()

bcApplyWest(da, A, b)
bcopy = b.copy()

pcbnn = PCBNN(A)

# Set initial guess
x.setRandom()
xnorm = b.dot(x)/x.dot(A*x)
x *= xnorm

ksp = PETSc.KSP().create()
ksp.setOperators(A)
ksp.setType(ksp.Type.PYTHON)
pyKSP = KSP_AMPCG(pcbnn)
pyKSP.callback = callback(da)
ksp.setPythonContext(pyKSP)
ksp.setInitialGuessNonzero(True)
ksp.setFromOptions()

ksp.solve(b, x)

viewer = PETSc.Viewer().createVTK(path + 'solution_3d.vts', 'w', comm = PETSc.COMM_WORLD)
x.view(viewer)

lamb_petsc = da.createGlobalVec()
lamb_a = da.getVecArray(lamb_petsc)
coords = da.getCoordinates()
coords_a = da.getVecArray(coords)
E = lame_coeff(coords_a[:, :, :, 0], coords_a[:, :, :, 1], coords_a[:, :, :, 2], E1, E2)
nu = lame_coeff(coords_a[:, :, :, 0], coords_a[:, :, :, 1], coords_a[:, :, :, 2], nu1, nu2)

lamb_a[:, :, :, 0] = (nu*E)/((1+nu)*(1-2*nu)) 
lamb_a[:, :, :, 1] = mpi.COMM_WORLD.rank
lamb_petsc.view(viewer)

viewer.destroy()

Overwriting demo_3d.py


In [7]:
!mpiexec -np 4 python demo_3d.py -AMPCG_verbose -ksp_monitor -PCBNN_verbose -PCBNN_Geneo

Subdomain number 3 contributes 6 coarse vectors as zero energy modes of local solver
Subdomain number 1 contributes 6 coarse vectors as zero energy modes of local solver
Subdomain number 0 contributes 0 coarse vectors as zero energy modes of local solver
Subdomain number 2 contributes 6 coarse vectors as zero energy modes of local solver
GenEO eigenvalue number 0 for lambdamax in subdomain 0: (0.00027210717238778715+0j)
GenEO eigenvalue number 1 for lambdamax in subdomain 0: (0.00028968623656886334+0j)
GenEO eigenvalue number 2 for lambdamax in subdomain 0: (0.004659628069890074+0j)
GenEO eigenvalue number 3 for lambdamax in subdomain 0: (0.004662790570237592+0j)
GenEO eigenvalue number 4 for lambdamax in subdomain 0: (0.007427946211460551+0j)
GenEO eigenvalue number 5 for lambdamax in subdomain 0: (0.0074445559391044846+0j)
GenEO eigenvalue number 6 for lambdamax in subdomain 0: (0.023991319480995324+0j)
GenEO eigenvalue number 7 for lambdamax in subdomain 0: (0.023992114566999734+0j)

 23 KSP Residual norm 3.321180687944e-06 
	natural_norm -> 2.14657371e-08
	ti -> 1.08733139e+00


In [8]:
from plot import plot_solution

In [10]:
plot_solution('output_3d', 'solution_3d.vts')

HBox(children=(Renderer(camera=PerspectiveCamera(position=(0.0, 0.0, 13.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), controls=[OrbitControls(controlling=PerspectiveCamera(position=(0.0, 0.0, 13.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)))], scene=Scene(background='black', children=(Mesh(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(1287900, 3), dtype=float32>, 'color': <BufferAttribute shape=(429300, 3, 3), dtype=float32>}), material=MeshLambertMaterial(alphaMap=None, aoMap=None, color='0xF5F5F5', emissiveMap=None, envMap=None, lightMap=None, map=None, opacity=0.5, side='DoubleSide', specularMap=None, transparent=True, vertexColors='VertexColors'), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), LineSegments(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(858600, 3), dtype=float32>}), material=LineBasicMaterial(color='0xffffff'), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), AmbientLight(quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0))), fog=None, overrideMaterial=None, position=(-5.0199076638509466, 0.028396277090850464, -0.5000000234667331), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), shadowMap=WebGLShadowMap()), VBox(children=(Dropdown(description='Fields', options=OrderedDict([('u', 0), ('v', 2), ('lambda', 3), ('rank', 4)]), value=0), Dropdown(description='domains', options=OrderedDict([('all', -1), ('domain 0', 0), ('domain 1', 1), ('domain 2', 2), ('domain 3', 3)]), value=-1), Dropdown(description='coarse vecs', options=OrderedDict([('none', -1), ('coarse vec 0', 0), ('coarse vec 1', 1), ('coarse vec 2', 2), ('coarse vec 3', 3), ('coarse vec 4', 4), ('coarse vec 5', 5), ('coarse vec 6', 6), ('coarse vec 7', 7), ('coarse vec 8', 8), ('coarse vec 9', 9), ('coarse vec 10', 10), ('coarse vec 11', 11), ('coarse vec 12', 12), ('coarse vec 13', 13), ('coarse vec 14', 14), ('coarse vec 15', 15), ('coarse vec 16', 16), ('coarse vec 17', 17), ('coarse vec 18', 18), ('coarse vec 19', 19), ('coarse vec 20', 20), ('coarse vec 21', 21), ('coarse vec 22', 22), ('coarse vec 23', 23), ('coarse vec 24', 24), ('coarse vec 25', 25), ('coarse vec 26', 26), ('coarse vec 27', 27), ('coarse vec 28', 28), ('coarse vec 29', 29), ('coarse vec 30', 30), ('coarse vec 31', 31), ('coarse vec 32', 32), ('coarse vec 33', 33), ('coarse vec 34', 34), ('coarse vec 35', 35), ('coarse vec 36', 36), ('coarse vec 37', 37), ('coarse vec 38', 38), ('coarse vec 39', 39), ('coarse vec 40', 40), ('coarse vec 41', 41), ('coarse vec 42', 42), ('coarse vec 43', 43), ('coarse vec 44', 44), ('coarse vec 45', 45), ('coarse vec 46', 46), ('coarse vec 47', 47), ('coarse vec 48', 48), ('coarse vec 49', 49), ('coarse vec 50', 50), ('coarse vec 51', 51), ('coarse vec 52', 52), ('coarse vec 53', 53), ('coarse vec 54', 54), ('coarse vec 55', 55), ('coarse vec 56', 56), ('coarse vec 57', 57), ('coarse vec 58', 58), ('coarse vec 59', 59), ('coarse vec 60', 60), ('coarse vec 61', 61), ('coarse vec 62', 62), ('coarse vec 63', 63), ('coarse vec 64', 64), ('coarse vec 65', 65), ('coarse vec 66', 66), ('coarse vec 67', 67), ('coarse vec 68', 68), ('coarse vec 69', 69)]), value=-1), Checkbox(value=True, description='Show displacement'), Checkbox(value=True, description='Show mesh'), IntSlider(value=100, continuous_update=False, description='Scale factor for coarse vec displacement', max=1000, min=1)))))

In [12]:
import plot

In [ ]:
plot.